In [ ]:
import sys
if '../' not in sys.path: sys.path.append("../")

import numpy as np
import amnet


def make_vgc(alpha):
    # inputs
    e_var = amnet.Variable(1, name='e')
    edot_var = amnet.Variable(1, name='edot')
    x = amnet.stack(e_var, edot_var)

    # affine transformations
    zero1 = amnet.atoms.make_const(np.zeros(1), x)
    ae    = amnet.AffineTransformation(np.array([[alpha, 0]]), x, np.zeros(1))
    e     = amnet.AffineTransformation(np.array([[1, 0]]), x, np.zeros(1))
    neg_e = amnet.AffineTransformation(np.array([[-1, 0]]), x, np.zeros(1))
    edot  = amnet.AffineTransformation(np.array([[0, 1]]), x, np.zeros(1))
    neg_edot = amnet.AffineTransformation(np.array([[0, -1]]), x, np.zeros(1))

    return amnet.atoms.make_or(
        amnet.atoms.make_or(
            zero1,
            ae,
            neg_e,
            neg_edot
        ),
        ae,
        e,
        edot
    )


def true_vgc(e, edot, alpha):
    if e * edot > 0:
        return alpha * e
    else:
        return 0


def test_vgc():
    alpha = 1.1
    phi_vgc = make_vgc(alpha)

    # test that true_vgc and test_vgc return the same evaluation
    for e in np.linspace(-2,2,12):
        for edot in np.linspace(-2,2,12):
            val1 = phi_vgc.eval(np.array([e, edot]))
            val2 = true_vgc(e, edot, alpha)
            if abs(val1 - val2) > 1e-10:
                print 'FAIL: (e,edot) = (%e, %e), val1=%e, val2=%e' % (e, edot, val1, val2)

test_vgc()

In [ ]:
# define VGC system
from __future__ import division
from numpy.polynomial import Polynomial
from scipy.linalg import expm
from numpy.linalg import inv

def c2d(Ac, Bc, Cc, Dc, h):
    n = Ac.shape[0]
    Ad = expm(h*Ac)
    Bd = np.dot(inv(Ac), Ad - np.eye(n))
    Cd = np.copy(Cc)
    Dd = np.copy(Dd)
    return (Ad, Bd, Cd, Dd)


def make_L(h):
    # plant transfer function
    p = 1.0
    P_num = Polynomial([1, 0.05])
    P_den = Polynomial([-p, 1.0]) * (Polynomial([1, 0.01]) ** 2)

    # linear controller transfer function
    k_p = 4.0
    f_lp = 10.0
    beta_lp = 0.7
    f_i = 0.5
    C_num = Polynomial([k_p*2.0*np.pi*f_i, k_p])
    C_den = Polynomial([0, 1]) * Polynomial([1, (2.0*beta_lp)/(2*np.pi*f_lp), 1.0/(2*np.pi*f_lp)**2 ])

    # linear loop transfer function L = PC
    PC_num = P_num * C_num
    PC_den = C_num * C_den
    #print (PC_num, PC_den)
    n = len(PC_den.coef) - 1 # dimension of ss(L)
    a = PC_den.coef;
    b = PC_num.coef;
    assert len(b) < len(a), 'L=PC must be strictly proper'
    assert abs(a[-1]) > 1e-10, 'L=PC is poorly conditioned'
    # controller canonical form
    Ac = np.concatenate((np.eye(n-1,n,1), (-1.0/a[n] * a[0:n]).reshape((1,n))), axis=0)
    Bc = np.zeros((n,1)); Bc[n-1] = 1.0/a[n]
    Cc = np.concatenate((b, np.zeros(n-len(b)))).reshape((1,n))    
    # sample continuous-time system
    return c2d(Ac, Bc, Cc, np.zeros((1,1)), h)

# discretization time
h = 0.01
(A, B, C, _) = make_L(h)




In [ ]:
p = 1.0
P_num = Polynomial([1, 0.05])
P_den = Polynomial([-p, 1.0]) * (Polynomial([1, 0.01]) ** 2)

# linear controller transfer function
k_p = 4.0
f_lp = 10.0
beta_lp = 0.7
f_i = 0.5
C_num = Polynomial([k_p*2.0*np.pi*f_i, k_p])
C_den = Polynomial([0, 1]) * Polynomial([1, (2.0*beta_lp)/(2*np.pi*f_lp), 1.0/(2*np.pi*f_lp)**2 ])

# linear loop transfer function L = PC
PC_num = P_num * C_num
PC_den = C_num * C_den
#print (PC_num, PC_den)
n = len(PC_den.coef) - 1 # dimension of ss(L)
a = PC_den.coef;
b = PC_num.coef;
assert len(b) < len(a), 'L=PC must be strictly proper'
assert abs(a[-1]) > 1e-10, 'L=PC is poorly conditioned'
# controller canonical form
Ac = np.concatenate((np.eye(n-1,n,1), (-1.0/a[n] * a[0:n]).reshape((1,n))), axis=0)
Bc = np.zeros((n,1)); Bc[n-1] = 1.0/a[n]
Cc = np.concatenate((b, np.zeros(n-len(b)))).reshape((1,n))    
# sample continuous-time system

In [ ]:
Ac